In [1]:
import pandas as pd
import numpy as np
import os
import sys
import xgboost as xgb


In [2]:
train_data = pd.read_csv(
    os.path.join(
        os.path.dirname(os.getcwd()),
        'data',
        'training',
        'final_anomaly_training.csv'
    )
)
train_data.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,5.000000e+05,5.000000e+05,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,7.620937e-01,1.327140e+00,5.115073e-01,-0.118658,0.350732,0.100730,-0.349274,0.087404
std,2.952638e+00,9.519952e+00,1.749233e+00,0.323386,0.477200,0.300971,0.476741,0.282426
min,-4.522426e-01,-3.267150e-01,-6.123311e-01,-1.000000,0.000000,0.000000,-1.000000,0.000000
25%,-2.770901e-01,-2.293651e-01,-3.214841e-01,0.000000,0.000000,0.000000,-1.000000,0.000000
50%,4.034820e-17,-1.820020e-17,3.426058e-17,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.229099e-01,7.706349e-01,6.785159e-01,0.000000,1.000000,0.000000,0.000000,0.000000
max,2.629377e+02,3.885243e+03,1.646674e+02,0.000000,1.000000,1.000000,0.000000,1.000000


In [4]:
fraud_count = train_data[train_data['fraud'] == 1].shape[0]
print(f"Total number of transactions: {train_data.shape[0]}")
print(f"Number of fraud cases: {fraud_count}")

Total number of transactions: 500000
Number of fraud cases: 43702


In [ ]:
# Prepare data for XGBoost anomaly detection (unsupervised)
X = train_data.drop(columns=['fraud'])

dtrain = xgb.DMatrix(X)

# Set up parameters for anomaly detection (isolation forest-like)
params = {
    'objective': 'binary:logistic',
    'tree_method': 'hist',
    'max_depth': 8,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'logloss'
}